In [1]:
import os
import cv2
import imageio
import numpy as np
from skimage.io import imread, imshow

In [2]:
#Set paths accordingly
segmentations_path = "C:\\Users\\SWONG\\PRS_project\\Data_Set\\val\\melanoma\\segmentation\\"
img_input_path = "C:\\Users\\SWONG\\PRS_project\\Data_Set\\val\\melanoma\\images\\"
img_output_path = "C:\\Users\\SWONG\\PRS_project\\Data_Set\\val\\melanoma\\border_cropped\\"

In [3]:
def get_perimeter(img, img_border, width, height):
#getting the image perimeter from the segmentation mask

    # getting border pixels in the left to right direction
    for i in range(height):
        for j in range(width - 1):
            try:
                if img[i][j] == 0 and img[i][j + 1] == 1:
                    img_border[i][j] = 1
                elif img[i][j] == 1 and img[i][j + 1] == 0:
                    img_border[i][j + 1] = 1
            except:
                print('Pixel at edge, moving on.')
                
    # getting border pixels in the right to left direction
    for i in range(height):
        for j in range(width - 1):
            try:
                if img[i][width - j - 1] == 0 and img[i][width - j - 2] == 1:
                    img_border[i][width - j - 1] = 1
                elif img[i][width - j - 1] == 1 and img[i][width - j - 2] == 0:
                    img_border[i][width - j - 2] = 1
            except:
                print('Pixel at edge, moving on.')

    # getting border pixels in the up to down direction
    for j in range(width):
        for i in range(height - 1):
            try:
                if img[i][j] == 0 and img[i + 1][j] == 1:
                    img_border[i][j] = 1
                elif img[i][j] == 1 and img[i + 1][j] == 0:
                    img_border[i + 1][j] = 1
            except:
                print('Pixel at edge, moving on.')

    # getting border pixels in the down to up direction
    for j in range(width):
        for i in range(height - 1):
            try:
                if img[height - i - 1][j] == 0 and img[height - i - 2][j] == 1:
                    img_border[height - i - 1][j] = 1
                elif img[height - i - 1][j] == 1 and img[height - i - 2][j] == 0:
                    img_border[height - i - 2][j] = 1
            except:
                print('Pixel at edge, moving on.')
                
    return img_border

In [4]:
def get_outward_pixels(img, img_border, width, height):
#get 5 pixels outwards, left right up down direction

    out_px = 5

    #Getting leftward 5 pixels
    for i in range(height):
        for j in range(width - 1):
            try:
                if img[i][j] == 0 and img[i][j + 1] == 1:
                    img_border[i][(j - out_px):j] = 1
            except:
                print('Pixel at edge, moving on.')

    #Getting rightward 5 pixels
    for i in range(height):
        for j in range(width - 1):
            try:
                if img[i][width - j - 1] == 0 and img[i][width - j - 2] == 1:
                    img_border[i][(width-j-1) : (width-j-1+out_px)] = 1
            except:
                print('Pixel at edge, moving on.')

    #Getting upward 5 pixels
    for j in range(width):
        for i in range(height - 1):
            try:
                if img[i][j] == 0 and img[i + 1][j] == 1:
                    for p in range(1, out_px+1):
                        img_border[i-p][j] = 1
            except:
                print('Pixel at edge, moving on.')
                
    #Getting downward 5 pixels
    for j in range(width):
        for i in range(height - 1):
            try:
                if img[height - i - 1][j] == 0 and img[height - i - 2][j] == 1:
                    for p in range(out_px):
                        img_border[height-i-1+p][j] = 1  
            except:
                print('Pixel at edge, moving on.')
                
    return(img_border)

In [5]:
def get_inward_pixels(img, img_border, width, height):
#get 20 pixels inwards, right left down up direction

    inPixels = 20

    #Getting rightward 20 pixels
    for i in range(height):
        for j in range(width - 1):
            try:
                if img[i][j] == 0 and img[i][j + 1] == 1:
                    img_border[i][j+1:j+inPixels+1] = 1
            except:
                print('Pixel at edge, moving on.')

    #Getting leftward 20 pixels
    for i in range(height):
        for j in range(width - 1):
            try:
                if img[i][width - j - 1] == 0 and img[i][width - j - 2] == 1:
                    img_border[i][(width-j-1-inPixels) : (width-j-1)] = 1
            except:
                print('Pixel at edge, moving on.')

    #Getting downward 20 pixels
    for j in range(width):
        for i in range(height - 1):
            try:
                if img[i][j] == 0 and img[i + 1][j] == 1:
                    for p in range(i+1, i+1+inPixels):
                        img_border[p][j] = 1
            except:
                print('Pixel at edge, moving on.')
                
    #Getting upward 20 pixels
    for j in range(width):
        for i in range(height - 1):
            try:
                if img[height - i - 1][j] == 0 and img[height - i - 2][j] == 1:
                    for p in range(inPixels):
                        img_border[height-i-2-p][j] = 1
            except:
                print('Pixel at edge, moving on.')
                
    return img_border

In [6]:
def create_border_mask(seg_name):

    seg = imread(segmentations_path + seg_name, as_gray=True)
    
    #make the pixel value binary on 1 color channel
    seg = seg/255
    width = seg.shape[1]
    height = seg.shape[0]

    # img_border will store the border imformation from the mask
    seg_border = np.empty(seg.shape, dtype=int)
    horizontal = seg.shape[1]
    vertical = seg.shape[0]
    
    seg_border = get_perimeter(seg, seg_border, width, height)
    seg_border = get_outward_pixels(seg, seg_border, width, height)
    seg_border = get_inward_pixels(seg, seg_border, width, height)
    
    return seg_border

In [7]:
def crop_border(img, img_crop, mask):
    height = img.shape[0]
    width = img.shape[1]
    
    for i in range(height):
        for j in range(width):
            if int(mask[i][j]) != 0:
                #to make the RGB channels correspond properly as original is loaded in BGR
                img_crop[i][j][0] = img[i][j][2]
                img_crop[i][j][1] = img[i][j][1]
                img_crop[i][j][2] = img[i][j][0]
            else:
                img_crop[i][j][:] = 0
    return img_crop

In [8]:
#apply the border cropping mask on images
img_list = os.listdir(img_input_path)

for image_name in img_list:
    seg_name = image_name.split('.')[0]+"_segmentation.png"
    
    #create the border mask
    seg_border = create_border_mask(seg_name)
        
    #load the original image
    img = cv2.imread(img_input_path + image_name)
    
    #create am empty nd array with same shape as img to store the cropped img
    img_crop = np.empty(img.shape, dtype=np.uint8)
    
    #apply the border cropping
    img_crop = crop_border(img, img_crop, seg_border)
    
    #save the border-cropped image to output path
    imageio.imwrite(img_output_path + image_name, img_crop)

Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edge, moving on.
Pixel at edg